In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import fuzzywuzzy.process

In [ ]:
df=pd.read_csv('/kaggle/input/zomato-dataset/zomato.csv')
df.head()

# Analysing and Cleaning of Data

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

### Droping unnecessary columns

In [ ]:
df.drop(columns=['url','reviews_list','menu_item','dish_liked','cuisines','phone','address','location'],inplace=True)
df.columns

### Droping duplicates

In [ ]:
df.drop_duplicates(ignore_index=True,inplace=True)

### renaming columns

In [ ]:
df.rename(columns={'approx_cost(for two people)':'_2platescost','listed_in(city)':'location','name':'restaurants'},inplace=True)
df.head()

## Analysing rate column

In [ ]:
df.rate.unique()

In [ ]:
# converting the rate dtype to float
df.rate=df['rate'].replace(["NEW","-"],np.nan)
for i in range(len(df.rate)):
    if df.loc[i,'rate'] is not np.nan:
        df.loc[i,'rate']= float(str(df.loc[i,'rate']).split('/')[0])
df.rate.unique()

In [ ]:
df.head()

### Handling null values in rate column

In [ ]:
df['rate'].isnull().sum()

In [ ]:
resturant_with_nan=df.loc[df['rate'].isnull(),'restaurants'].unique()
resturant_with_nan

In [ ]:
#providing rating by calculating avg rating of other branches of the same resturant
for i in range(len(resturant_with_nan)):
    rest=df.loc[df["restaurants"]==resturant_with_nan[i],'rate']
    df.loc[rest[rest.isnull()].index,'rate']=round(rest.mean(),1)
    #print(df.loc[df["restaurants"]==resturant_with_nan[i],'rate'])
    #print("*"*30)

In [ ]:
df['rate'].isnull().sum()

In [ ]:
df.rate.fillna(df.rate.mean(),inplace=True)# since many resturants where not rated, I have provided the average rating

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

## Analyzing rest_type column

In [ ]:
len(df.rest_type.unique())

In [ ]:
df.rest_type.unique()

noticed some duplicate rest_type values like 'Cafe, Casual Dining' and 'Casual Dining, Cafe'
or 'Quick Bites, Cafe' and 'Cafe, Quick Bites', lets focus on cleaning these duplications as it occurs only to the resturants that have 2 rest_type categories combined

In [ ]:
lst=[]
for i in df.rest_type.unique():
    if pd.notna(i)and ',' in i:
        lst.append(i)
lst

In [ ]:
def replace_string(dt,col,str_2_match,min_ratio=90):
    match = fuzzywuzzy.process.extract(str_2_match,dt[col].unique(),limit=10,scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches=[i[0] for i in match if i[1]>min_ratio]
    match_index=dt[col].isin(close_matches)
    dt.loc[match_index,col]=str_2_match

In [ ]:
for i in lst:
    replace_string(df,'rest_type',i,min_ratio=90)

In [ ]:
len(df.rest_type.unique()) # the uniqueness has been reduced from 94 to 69 after removing such duplications

In [ ]:
df.rest_type.unique()

### Handling null values in rest_type column

I have used same method as 'rate' column to handle null value in 'rest_type' column but instead of average I have taken mode value, as string values don't have mean

In [ ]:
null_rest_type=df.loc[df['rest_type'].isnull(),'restaurants'].unique()
null_rest_type

In [ ]:
for i in range(len(null_rest_type)):
    typ=df.loc[df['restaurants']==null_rest_type[i],'rest_type']
    if typ.mode().empty:
        pass
    else:
        df.loc[typ[typ.isnull()].index,'rest_type']=typ.mode().iloc[0]

In [ ]:
df.rest_type.fillna(df['rest_type'].mode()[0],inplace=True)

In [ ]:
df.isnull().sum()

## Analysing _2platescost column

In [ ]:
df['_2platescost'].unique()

In [ ]:
## converting the _2platescost dtype to float
def float_conv(i):
    if ',' in str(i):
        return float(i.replace(',',''))
    else:
        return float(i)
df._2platescost=df._2platescost.apply(float_conv)
df['_2platescost'].unique()

### Handling null values in _2platescost column

In [ ]:
df.loc[pd.isnull(df['_2platescost']),'restaurants'].unique()

In [ ]:
null_2platescost=df.loc[df['_2platescost'].isnull(),'restaurants'].unique()
null_2platescost

In [ ]:
for i in range(len(null_2platescost)):
    _2plates=df.loc[df['restaurants']==null_2platescost[i],'_2platescost']
    df.loc[_2plates[_2plates.isnull()].index,'_2platescost']=round(_2plates.mean(),1)

In [ ]:
df._2platescost.fillna(df['_2platescost'].mean(),inplace=True)

In [ ]:
df.isnull().sum()

# Data Visualisation

In [ ]:
sns.set_style('dark')

In [ ]:
plt.figure(figsize=(7,7))
sns.distplot(df['rate'])

In [ ]:
#rating counts of all the resturants in a location that has over 2.5K resturants

In [ ]:
df.groupby('location')['restaurants'].size().sort_values(ascending=False) # top 6 locations have over 2.5K resutrants

In [ ]:
loc_index=df.groupby('location')['restaurants'].size().sort_values(ascending=False).iloc[0:6].index
f,a=plt.subplots(2,3,figsize=(15,10),dpi=100)
for i,axes_subplot in enumerate(a.flatten()):
    sns.histplot(data=df,x=df.loc[df["location"]==loc_index[i],'rate'],bins=6,ax=axes_subplot)
    axes_subplot.set_title(f"{i+1} {loc_index[i]}")
plt.show()

In [ ]:
plt.figure(figsize=(20,7))
df['rest_type'].value_counts().nlargest(30).plot(kind='bar',color='gold')

In [ ]:
# accept online orders, accept booking table, accept both, don't accept both
accept_only_online_orders = df.online_order.value_counts().loc['Yes'] - df[(df['online_order']=='Yes') & (df['book_table']=='Yes')].book_table.value_counts().loc['Yes']
#accept booking table
accept_only_booking_table = df.book_table.value_counts().loc['Yes']-df[(df['online_order']=='Yes') & (df['book_table']=='Yes')].book_table.value_counts().loc['Yes']
#accept both
accept_both = df[(df['online_order']=='Yes') & (df['book_table']=='Yes')].book_table.value_counts().loc['Yes']
#don't accept both
dont_accept_both = df[(df['online_order']=='No') & (df['book_table']=='No')].book_table.value_counts().loc['No']

size=[accept_only_online_orders,accept_only_booking_table,accept_both,dont_accept_both]
labels=['Accept only Online orders','Accept only Booking table','Accept Both',"Don't Accept both"]
colors=['gold','orange','coral','olive']
explode=(.01,.01,.01,.01)
plt.figure(figsize=(4,4),dpi=100)
plt.pie(size,autopct='%.1f%%',colors=colors,labels=labels,explode=explode)
plt.axis('equal')
plt.show()

In [ ]:
loc=df.groupby('location')['restaurants'].unique()
for i in range(loc.size):
    loc.iloc[i]=loc.iloc[i].size
loc=pd.DataFrame(loc)
loc.restaurants.astype(int)
loc

In [ ]:
loc.sort_values(by='restaurants',ascending=False).plot.bar(figsize=(20,6),color='gold')
plt.gcf().autofmt_xdate()
plt.show()